In [1]:
import numpy as np
import torch

from train import ALADTrainer
from preprocess import get_svhn

In [2]:
class Args:
    num_epochs=100
    lr=0.0002
    latent_dim=100
    normal_class=1
    batch_size=32
    pretrained=False
    spec_norm=False
    
    
args = Args()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = get_svhn(args)

alad = ALADTrainer(args, data, device)

Using downloaded and verified file: ./data/svhn/train_32x32.mat


In [ ]:
alad.train()

11088/11088: [===============================>] - ETA 0.4sss
Training... Epoch: 0, Discrimiantor Loss: 1.982, Generator Loss: 21.107
10560/11088: [==============================>.] - ETA 1.2ss

In [ ]:
import torch

from torch.autograd import Variable

def eval(model):
    labels = []
    scores = []

    model.G.eval()
    model.Dxx.eval()
    model.E.eval()

    with torch.no_grad():
        for img, label in data[1]:
            img = img.float().to(device)
            _, feature_real = model.Dxx(img, img)
            _, feature_gen = model.Dxx(img, model.G(model.E(img)))
            score = torch.sum(torch.abs(feature_real - feature_gen), dim=1)
            scores.append(score.cpu())
            labels.append(label.cpu())
    scores = torch.cat(scores, dim=0)
    labels = torch.cat(labels, dim=0)
    return labels, scores

In [ ]:
labels, scores = eval(alad)

In [ ]:
from sklearn.metrics import precision_recall_curve, roc_auc_score, auc

precision, recall, thresholds = precision_recall_curve(labels, scores)

print('ROC AUC score: {:.2f}'.format(roc_auc_score(labels, scores)*100))

In [ ]:
import matplotlib.pyplot as plt

plt.hist(scores[labels==0], color='r', alpha = 0.3)
plt.hist(scores[labels==1], color='b', alpha=0.3)
plt.show()